
## PARTE 0: CONFIGURAÇÃO DO AMBIENTE
 

In [1]:
# 1. Instalação de bibliotecas usando o comando mágico %pip
# Garante que a instalação ocorra no kernel correto do notebook.
%pip install -q pandas numpy pyarrow fastparquet matplotlib seaborn

# 2. Importação das bibliotecas
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Configuração de visualização do Pandas
# Garante que todas as colunas de um DataFrame sejam exibidas
pd.set_option('display.max_columns', None)

# Configura o estilo dos gráficos para um visual mais agradável
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

print("Ambiente configurado e bibliotecas importadas com sucesso!")
print(f"Versão do Pandas: {pd.__version__}")

Note: you may need to restart the kernel to use updated packages.
Ambiente configurado e bibliotecas importadas com sucesso!
Versão do Pandas: 2.3.3



## PARTE 1: CARGA E ESTRUTURAÇÃO DOS DADOS (ETL)


In [2]:
# --- 1.1 Carregando os dados do SINASC (Nascidos Vivos) ---
print("Iniciando a carga dos dados do SINASC...")

# ETAPA 1: Encontrar todos os caminhos dos arquivos .parquet
# O padrão 'sinasc_2020_2022/**/*.parquet' busca recursivamente em todas as subpastas.
parquet_path = 'sinasc_2020_2022/**/*.parquet'
list_parqutet_files = glob.glob(parquet_path, recursive=True)
print(f"Encontrados {len(list_parqutet_files)} arquivos Parquet.")
# Vamos exibir os 3 primeiros caminhos para verificar se estão corretos.
print("Exemplo de caminhos encontrados:")
print(list_parqutet_files[:3])

# ETAPA 2: Iterar, ler cada arquivo, extrair metadados e armazenar em uma lista
list_dfs = []

for file_path in list_parqutet_files:
    temp_df = pd.read_parquet(file_path)   # Lê o arquivo Parquet em um DataFrame temporário
    path_parts = file_path.split(os.sep) # Divide o caminho em partes
    dir_name = path_parts[-2]  # Nome do diretório (ano)
    year_collect = int(dir_name[-4:]) # Extrai o ano dos últimos 4 caracteres
    uf_collect = dir_name[2:4]   # Extrai a UF dos caracteres na posição 2 e 3

    temp_df['ANO_COLETA'] = year_collect  # Adiciona a coluna do ano de coleta
    temp_df['UF_COLETA'] = uf_collect    # Adiciona a coluna da UF de coleta

    list_dfs.append(temp_df)  # Adiciona o DataFrame temporário à lista

# ETAPA 3: Concatenar todos os DataFrames em um único DataFrame
if list_dfs:
    df_sinasc = pd.concat(list_dfs, ignore_index=True)
    print("\nDados do SINASC carregados e consolidados com sucesso!")
    print(f"Total de registros no DataFrame final: {len(df_sinasc)}")
    print(f"Total de colunas: {len(df_sinasc.columns)}")

    display(df_sinasc.head())
else:
    print("\nErro: Nenhum DataFrame foi carregado. Verifique os arquivos Parquet.")

Iniciando a carga dos dados do SINASC...
Encontrados 984 arquivos Parquet.
Exemplo de caminhos encontrados:
['sinasc_2020_2022/DNSP2022/part-00001-bd6c59fe-baea-40ee-a750-b5ab8e63d89a-c000.gz.parquet', 'sinasc_2020_2022/DNSP2022/part-00003-bd6c59fe-baea-40ee-a750-b5ab8e63d89a-c000.gz.parquet', 'sinasc_2020_2022/DNSP2022/part-00010-bd6c59fe-baea-40ee-a750-b5ab8e63d89a-c000.gz.parquet']

Dados do SINASC carregados e consolidados com sucesso!
Total de registros no DataFrame final: 3999785
Total de colunas: 21


,CODMUNNASC,LOCNASC,CODMUNRES,DTNASC,SEXO,RACACOR,PESO,IDADEMAE,ESTCIVMAE,ESCMAE,GRAVIDEZ,CONSULTAS,RACACORMAE,DTNASCMAE,GESTACAO,SEMAGESTAC,CONSPRENAT,PARTO,CODESTAB,ANO_COLETA,UF_COLETA
0,355240,1,355240,30032022,2,4,2600,33,1,4,1,3,4,19021989,4,36,05,2,2083981,2022,SP
1,355030,1,355030,29062022,2,4,3800,36,2,3,1,4,4,03061986,5,41,16,1,7711980,2022,SP
2,355710,1,355710,21062022,2,1,3035,36,2,4,1,4,1,22021986,5,38,10,2,2081377,2022,SP
3,355210,1,355210,29042022,2,1,3420,34,1,5,1,2,1,18041988,5,38,03,2,2079704,2022,SP
4,355370,1,355370,27062022,1,4,2880,22,1,4,1,4,4,02101999,5,38,11,2,2078295,2022,SP


## ESTRATÉGIA DE OTIMIZAÇÃO: REDUÇÃO DO USO DE MEMÓRIA

In [3]:
# ==============================================================================
# OTIMIZAÇÃO DE MEMÓRIA
# ==============================================================================

# Esta função será nossa ferramenta para reduzir o uso de memória dos DataFrames.

def optimize_memory_usage(df, print_log=True):
    """
    Itera sobre todas as colunas de um DataFrame e modifica o tipo de dado
    para o formato mais eficiente em memória.

    Args:
    df (pd.DataFrame): O DataFrame a ser otimizado.
    print_log (bool): Se True, imprime o log da redução de memória.

    Returns:
    pd.DataFrame: O DataFrame otimizado.
    """
    if print_log:
        # Calcula o uso de memória inicial
        mem_usage_before = df.memory_usage(deep=True).sum() / 1024**2
        print(f"Uso de memória inicial: {mem_usage_before:.2f} MB")
    
    for col in df.columns:
        col_type = df[col].dtype

        # Verifica se a coluna é numérica (inteiro ou float)
        if pd.api.types.is_numeric_dtype(col_type) and not pd.api.types.is_datetime64_any_dtype(df[col]):
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Se for do tipo inteiro, tenta diminuir para o menor inteiro possível
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            # Se for do tipo float, tenta diminuir para um float menor
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        
        # Se a coluna for de texto (object)
        elif col_type == 'object':
            # Se a proporção de valores únicos for baixa, converte para 'category'
            # O tipo 'category' é muito mais eficiente para strings repetidas (ex: 'UF_COLETA')
            if df[col].nunique() / len(df[col]) < 0.5:
                df[col] = df[col].astype('category')
    
    if print_log:
        # Calcula o uso de memória final
        mem_usage_after = df.memory_usage(deep=True).sum() / 1024**2
        reduction = 100 * (mem_usage_before - mem_usage_after) / mem_usage_before
        print(f"Uso de memória final: {mem_usage_after:.2f} MB ({reduction:.2f}% de redução)")
        
    return df

print("Função 'optimize_memory_usage' definida com sucesso.")

Função 'optimize_memory_usage' definida com sucesso.


In [4]:
# --- Aplicando a otimização nos nossos DataFrames ---

print("Otimizando df_sinasc...")
# Chamamos a função e sobrescrevemos a variável original com a versão otimizada
df_sinasc = optimize_memory_usage(df_sinasc)

# Faremos o mesmo para o df_ibge_cities quando o carregarmos.
# Por enquanto, garantimos que o df_sinasc está leve.

Otimizando df_sinasc...
Uso de memória inicial: 4605.44 MB
Uso de memória final: 199.19 MB (95.67% de redução)



## PARTE 1.2: CARREGANDO DADOS DO IBGE E ENRIQUECENDO O DF_SINASC


In [5]:
# --- 1.2.1 Carregando os dados de municípios do IBGE ---
print("Carregando os dados de municípios do IBGE...")

url_ibge_cities = 'https://raw.githubusercontent.com/leogermani/estados-e-municipios-ibge/master/municipios.csv'
df_ibge_cities = pd.read_csv(url_ibge_cities)

# Renomeando as colunas para um padrão sem espaços e maiúsculo para consistência
df_ibge_cities.rename(columns={
    'COD UF': 'COD_UF',
    'COD': 'COD_MUN',
    'NOME': 'NOME_MUN',
}, inplace=True)

# OTIMIZAÇÃO DE MEMÓRIA PARA df_ibge_cities
df_ibge_cities = optimize_memory_usage(df_ibge_cities)

print("Base de municípios do IBGE carregados com sucesso!")
display(df_ibge_cities.head())

Carregando os dados de municípios do IBGE...
Uso de memória inicial: 0.50 MB
Uso de memória final: 0.48 MB (4.89% de redução)
Base de municípios do IBGE carregados com sucesso!


,COD_UF,COD_MUN,NOME_MUN
0,11,1100015,Alta Floresta D'oeste
1,11,1100023,Ariquemes
2,11,1100031,Cabixi
3,11,1100049,Cacoal
4,11,1100056,Cerejeiras


## Preparando o df_sinasc para o "join"

In [ ]:
# --- 1.2.2 Preparando o df_sinasc para o join ---
print("\nPreparando o DataFrame do SINASC para o enriquecimento...")

# ETAPA DE LIMPEZA:
# Primeiro, convertemos toda a coluna 'CODMUNNASC' para string, tratando erros.
df_sinasc['CODMUNNASC_STR'] = df_sinasc['CODMUNNASC'].astype(str)

# expressão regular para extrair APENAS a sequência de dígitos.
df_sinasc['CODMUNNASC_CLEANED'] = df_sinasc['CODMUNNASC_STR'].str.extract('(\\d+)').fillna('')

# Agora que temos uma coluna limpa, podemos extrair os 2 primeiros dígitos com segurança.
df_sinasc['COD_UF_NASC'] = df_sinasc['CODMUNNASC_CLEANED'].str[:2]

# Verificando as novas colunas criadas
print("Colunas de limpeza e 'COD_UF_NASC' criadas no df_sinasc.")
display(df_sinasc[['CODMUNNASC', 'CODMUNNASC_STR', 'CODMUNNASC_CLEANED', 'COD_UF_NASC']].sample(10))


Preparando o DataFrame do SINASC para o enriquecimento...
Colunas de limpeza e 'COD_UF_NASC' criadas no df_sinasc.


,CODMUNNASC,CODMUNNASC_STR,CODMUNNASC_CLEANED,COD_UF_NASC
3929156,521250,521250,521250,52
1330369,530010,530010,530010,53
1916948,260620,260620,260620,26
465085,432140,432140,432140,43
390020,292530,292530,292530,29
3861707,314330,314330,314330,31
625343,130260,130260,130260,13
2004568,520870,520870,520870,52
143611,350280,350280,350280,35
2378039,354980,354980,354980,35


: 

## Executando o join (merge)

In [ ]:
# --- 1.2.3 Executando o join (merge) ---
print("\nIniciando o merge entre as bases SINASC e IBGE...")


# Precisamos que as chaves sejam do mesmo tipo de dado; Converteremos nossa nova coluna limpa para numérico, tratando erros.
df_sinasc['COD_UF_NASC'] = pd.to_numeric( df_sinasc['COD_UF_NASC'], errors='coerce' )

# Usamos um 'left' join para manter todos os registros do SINASC
df_sinasc_enriched = pd.merge( 
    df_sinasc,
    df_ibge_cities,
    left_on='COD_UF_NASC',
    right_on='COD_UF',
    how='left'
)

# Limpeza final: removemos as colunas temporárias usadas no processo
df_sinasc_enriched.drop(columns=['CODMUNNASC_STR', 'CODMUNNASC_CLEANED'], inplace=True)

print("Merge concluído com sucesso!")
print(f"Número de colunas após o merge: {len(df_sinasc_enriched.columns)}")

# Exibindo as colunas relevantes para verificar o sucesso do join
display(df_sinasc_enriched[['CODMUNNASC', 'COD_UF_NASC', 'COD_UF', 'NOME_MUN']].head())


Iniciando o merge entre as bases SINASC e IBGE...
